In [1]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, plot_roc_curve, confusion_matrix

# Import data and create datasets

As labels dos dados estão definidas tais que:
- Background: 0
- Sinal: 1

In [2]:
# Gather data
bkgd = pd.read_hdf("Data/Background.h5", key='bkgd')
fcnc = pd.read_hdf("Data/FCNC.h5", key="fcnc")
X_train = pd.concat([bkgd, fcnc])
X_train.describe()

,Electron1_Eta,Electron1_PT,Electron1_Phi,Electron2_Eta,Electron2_PT,Electron2_Phi,Electron_Multi,FatJet1_Eta,FatJet1_Mass,FatJet1_PT,...,Muon1_Eta,Muon1_PT,Muon1_Phi,Muon2_Eta,Muon2_PT,Muon2_Phi,Muon_Multi,ScalarHT_HT,gen_weights,Label
count,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,...,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06,4.440800e+06
mean,4.642362e-06,4.102770e+01,1.146982e-03,3.100396e-06,1.036529e+01,-4.581339e-06,8.324586e-01,3.533756e-04,8.693247e+00,3.873254e+01,...,3.026105e-04,5.423478e+01,-1.956640e-04,-5.599270e-05,1.706459e+01,-1.866145e-05,1.196289e+00,3.714505e+02,1.064602e-04,4.407427e-02
std,8.472479e-01,5.473847e+01,1.371933e+00,5.798893e-01,2.252014e+01,9.181378e-01,8.186355e-01,4.162720e-01,2.652501e+01,1.099466e+02,...,9.976884e-01,5.717222e+01,1.571230e+00,7.815677e-01,2.605593e+01,1.198585e+00,8.223965e-01,2.021479e+02,5.091671e-04,2.052602e-01
min,-2.499997e+00,0.000000e+00,-3.141592e+00,-2.499985e+00,0.000000e+00,-3.141592e+00,0.000000e+00,-4.000467e+00,-6.103516e-05,0.000000e+00,...,-2.399990e+00,0.000000e+00,-3.141592e+00,-2.399981e+00,0.000000e+00,-3.141588e+00,0.000000e+00,4.447565e+01,3.264224e-09,0.000000e+00
25%,-1.986022e-01,0.000000e+00,-3.912720e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-5.680689e-01,1.020456e+01,-1.047653e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.342856e+02,5.842474e-07,0.000000e+00
50%,0.000000e+00,2.898474e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,4.799751e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,3.308271e+02,2.374618e-05,0.000000e+00
75%,2.005566e-01,6.450357e+01,3.971428e-01,0.000000e+00,1.263088e+01,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.684932e-01,7.523514e+01,1.046757e+00,0.000000e+00,3.088431e+01,0.000000e+00,2.000000e+00,4.581491e+02,2.458480e-05,0.000000e+00
max,2.499979e+00,2.124003e+03,3.141593e+00,2.499981e+00,9.984486e+02,3.141575e+00,5.000000e+00,3.873402e+00,7.324354e+02,2.939618e+03,...,2.399990e+00,2.646508e+03,3.141587e+00,2.399961e+00,1.217265e+03,3.141591e+00,5.000000e+00,4.926052e+03,3.651510e-03,1.000000e+00


In [3]:
fcnc.describe()

,Electron1_Eta,Electron1_PT,Electron1_Phi,Electron2_Eta,Electron2_PT,Electron2_Phi,Electron_Multi,FatJet1_Eta,FatJet1_Mass,FatJet1_PT,...,Muon1_Eta,Muon1_PT,Muon1_Phi,Muon2_Eta,Muon2_PT,Muon2_Phi,Muon_Multi,ScalarHT_HT,gen_weights,Label
count,195725.000000,195725.000000,195725.000000,195725.000000,195725.000000,195725.000000,195725.000000,195725.000000,195725.000000,195725.000000,...,195725.000000,195725.000000,195725.000000,195725.000000,195725.000000,195725.000000,195725.000000,195725.000000,1.957250e+05,195725.0
mean,-0.002376,93.874797,0.002506,-0.001356,24.843815,-0.000378,1.003965,-0.000878,39.842755,214.031343,...,-0.002890,122.098796,0.008965,-0.002141,36.443064,0.000233,1.373611,608.585528,3.264224e-09,1.0
std,0.956811,135.176460,1.411901,0.677618,50.790100,1.049370,0.959648,0.882671,48.378385,246.652396,...,1.084205,147.214662,1.571437,0.853159,58.101179,1.274687,1.002467,371.896309,6.990108e-21,0.0
min,-2.499965,0.000000,-3.141324,-2.499727,0.000000,-3.141587,0.000000,-3.494491,-0.000061,0.000000,...,-2.399990,0.000000,-3.141585,-2.399888,0.000000,-3.141572,0.000000,51.668472,3.264224e-09,1.0
25%,-0.343061,0.000000,-0.539507,0.000000,0.000000,0.000000,0.000000,-0.127633,0.000000,0.000000,...,-0.684100,0.000000,-1.031103,0.000000,0.000000,0.000000,0.000000,354.401245,3.264224e-09,1.0
50%,0.000000,43.226986,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,9.294422,216.051682,...,0.000000,79.353065,0.000000,0.000000,0.000000,0.000000,1.000000,512.929443,3.264224e-09,1.0
75%,0.332078,138.506805,0.549541,0.000000,32.493484,0.000000,2.000000,0.119289,91.306366,354.365540,...,0.672683,175.863571,1.061248,0.000000,53.818260,0.000000,2.000000,753.832581,3.264224e-09,1.0
max,2.499960,2124.003174,3.141387,2.499523,998.448608,3.141527,5.000000,3.708754,732.435364,2939.618408,...,2.399966,2646.508301,3.141582,2.399788,1217.265381,3.141448,5.000000,4926.052246,3.264224e-09,1.0


In [4]:
bkgd.describe()

,Electron1_Eta,Electron1_PT,Electron1_Phi,Electron2_Eta,Electron2_PT,Electron2_Phi,Electron_Multi,FatJet1_Eta,FatJet1_Mass,FatJet1_PT,...,Muon1_Eta,Muon1_PT,Muon1_Phi,Muon2_Eta,Muon2_PT,Muon2_Phi,Muon_Multi,ScalarHT_HT,gen_weights,Label
count,4.245075e+06,4.245075e+06,4.245075e+06,4.245075e+06,4.245075e+06,4.245075e+06,4.245075e+06,4.245075e+06,4.245075e+06,4.245075e+06,...,4.245075e+06,4.245075e+06,4.245075e+06,4.245075e+06,4.245075e+06,4.245075e+06,4.245075e+06,4.245075e+06,4.245075e+06,4245075.0
mean,1.144244e-04,3.859111e+01,1.084335e-03,6.578391e-05,9.697738e+00,1.264110e-05,8.245510e-01,4.101399e-04,7.257056e+00,3.065015e+01,...,4.498187e-04,5.110581e+01,-6.180218e-04,4.014672e-05,1.617111e+01,-3.028545e-05,1.188113e+00,3.605171e+02,1.113686e-04,0.0
std,8.418525e-01,4.644635e+01,1.370063e+00,5.749829e-01,2.003727e+01,9.116318e-01,8.106689e-01,3.812480e-01,2.411029e+01,9.142469e+01,...,9.935177e-01,4.688299e+01,1.571220e+00,7.781081e-01,2.316157e+01,1.194959e+00,8.122001e-01,1.834634e+02,5.202477e-04,0.0
min,-2.499997e+00,0.000000e+00,-3.141592e+00,-2.499985e+00,0.000000e+00,-3.141592e+00,0.000000e+00,-4.000467e+00,-5.285799e-05,0.000000e+00,...,-2.399985e+00,0.000000e+00,-3.141592e+00,-2.399981e+00,0.000000e+00,-3.141588e+00,0.000000e+00,4.447565e+01,1.304437e-07,0.0
25%,-1.928962e-01,0.000000e+00,-3.839418e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,-5.635411e-01,1.028139e+01,-1.048443e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.310312e+02,5.842474e-07,0.0
50%,0.000000e+00,2.859756e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,4.753570e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,3.254956e+02,2.374618e-05,0.0
75%,1.953454e-01,6.321899e+01,3.898618e-01,0.000000e+00,1.121624e+01,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,5.642138e-01,7.347187e+01,1.046109e+00,0.000000e+00,3.038062e+01,0.000000e+00,2.000000e+00,4.480336e+02,2.458480e-05,0.0
max,2.499979e+00,1.451913e+03,3.141593e+00,2.499981e+00,5.076544e+02,3.141575e+00,5.000000e+00,3.873402e+00,5.243948e+02,1.723009e+03,...,2.399990e+00,1.249431e+03,3.141587e+00,2.399961e+00,6.135786e+02,3.141591e+00,4.000000e+00,3.903838e+03,3.651510e-03,0.0


In [5]:
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_train.drop(columns="Label"), X_train["Label"], 
                                                    test_size=1/3, random_state=42)

In [6]:
print(f"Train_size: {len(y_train)} | Test_size: {len(y_test)}")

Train_size: 2960533 | Test_size: 1480267


In [7]:
# Get sample weights
train_weights = X_train["gen_weights"]
test_weights = X_test["gen_weights"]

# Get training features
X_train = X_train.drop(["gen_weights"], axis=1)

# Get testing features
X_test = X_test.drop(["gen_weights"], axis=1)

# Get class weights
# Class 0 (background) is the reference class, with weight 1
class_weights = {
    0: 1,
    1: len(y_train[y_train == 0]) / len(y_train[y_train == 1])
}

In [8]:
print(f"Class weights:\n{class_weights}\n")
print(f"Train_weights:\n{train_weights.values}\n")
print(f"Test_weights:\n{test_weights.values}\n")
print(f"X_train:\n{X_train.values}\n")
print(f"y_train:\n{y_train.values}\n")
print(f"X_test:\n{X_test.values}\n")
print(f"y_test:\n{y_test.values}\n")

Class weights:
{0: 1, 1: 21.714099386983175}

Train_weights:
[5.84247368e-07 2.45847956e-05 5.84247368e-07 ... 2.37461767e-05
 3.26422429e-09 2.45847956e-05]

Test_weights:
[5.84247368e-07 2.37461767e-05 2.37461767e-05 ... 5.84247368e-07
 2.45847956e-05 5.84247368e-07]

X_train:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -2.43416023e+00
   2.00000000e+00  3.67212250e+02]
 [ 2.35044792e-01  6.95409927e+01  2.78545356e+00 ...  0.00000000e+00
   1.00000000e+00  3.25054840e+02]
 [-1.08409858e+00  2.43246975e+01 -2.79117322e+00 ...  0.00000000e+00
   1.00000000e+00  2.02325790e+02]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.72923017e+00
   2.00000000e+00  1.97660004e+02]
 [ 1.46371841e+00  3.39957458e+02  4.40391511e-01 ...  0.00000000e+00
   0.00000000e+00  1.05735388e+03]
 [ 5.15887201e-01  2.04379234e+01  2.43531656e+00 ...  0.00000000e+00
   1.00000000e+00  2.42583435e+02]]

y_train:
[0 0 0 ... 0 1 0]

X_test:
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+

In [9]:
print(f"X_train: {X_train.values.shape} | y_train: {y_train.values.shape} | train_weights: {train_weights.values.shape}")
print(f"X_test: {X_test.values.shape} | y_test: {y_test.values.shape} | test_weights: {test_weights.values.shape}")

X_train: (2960533, 69) | y_train: (2960533,) | train_weights: (2960533,)
X_test: (1480267, 69) | y_test: (1480267,) | test_weights: (1480267,)


# Build the model

In [10]:
class Standardization(keras.layers.Layer):
    def adapt(self, data_samples):
        self.means = np.mean(data_samples, axis=0, keepdims=True)
        self.stds = np.std(data_samples, axis=0, keepdims=True)
        
    def call(self, inputs):
        return (inputs - self.means) / (self.stds + keras.backend.epsilon())
    
std_layer = Standardization()
std_layer.adapt(X_train.values)

Tentar um modelo com menos complexidade e utilizar MC Dropout para tentar obter melhores resultados

In [11]:
inputs = keras.Input(shape=(69,))
standardize = std_layer(inputs)
fc1 = keras.layers.Dense(100, activation='relu')(inputs)
d1 = keras.layers.Dropout(0.1)(fc1, training=True)
fc2 = keras.layers.Dense(80, activation='relu')(d1)
d2 = keras.layers.Dropout(0.1)(fc2, training=True)
fc3 = keras.layers.Dense(60, activation='relu')(d2)
d3 = keras.layers.Dropout(0.1)(fc3, training=True)
fc4 = keras.layers.Dense(40, activation='relu')(d3)
outputs = keras.layers.Dense(1, activation='sigmoid')(fc4)
model = keras.Model(inputs, outputs)
model.compile(optimizer='adam', loss="binary_crossentropy",
              metrics=["accuracy", keras.metrics.AUC()])

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 69)]              0         
_________________________________________________________________
dense (Dense)                (None, 100)               7000      
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 80)                8080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 60)                4860      
_________________________________________________________________
dropout_2 (Dropout)          (None, 60)                0     

In [13]:
# Model name
name = "HiddenDense:100,80,60,40|BatchS:256|Dropout:0.1"

# Callbacks 

In [14]:
# Tensorboard
TB = keras.callbacks.TensorBoard("logs/" + name, write_images=True)

# Early Stopping
ES = keras.callbacks.EarlyStopping(monitor="val_loss", patience=12, verbose=2, mode="min")

# Model Checkpoint
MC = keras.callbacks.ModelCheckpoint("models/" + name + ".h5", save_best_only=True, monitor="val_loss",
                                     mode="min")

# Treinar modelo

O modelo tem uma AUC consideravelmente baixa

In [15]:
start = time.time()
history = model.fit(X_train.values, y_train.values, batch_size=256,
                    epochs=50, shuffle=True, validation_split=0.5,
                    sample_weight=train_weights.values, class_weight=class_weights, 
                    callbacks=[TB, ES, MC])
end = time.time()
print(f"The training took {(end - start)/3600} hours")

Epoch 1/50
5783/5783 [==============================] - 116s 20ms/step - loss: 5.3630e-07 - accuracy: 0.9559 - auc: 0.4998 - val_loss: 1.8017e-08 - val_accuracy: 0.9557 - val_auc: 0.5059
Epoch 2/50
5783/5783 [==============================] - 115s 20ms/step - loss: 8.1515e-08 - accuracy: 0.9562 - auc: 0.5078 - val_loss: 1.4364e-08 - val_accuracy: 0.9557 - val_auc: 0.5305
Epoch 3/50
5783/5783 [==============================] - 116s 20ms/step - loss: 5.4112e-08 - accuracy: 0.9562 - auc: 0.5197 - val_loss: 1.0438e-08 - val_accuracy: 0.9557 - val_auc: 0.5244
Epoch 4/50
5783/5783 [==============================] - 117s 20ms/step - loss: 4.7540e-08 - accuracy: 0.9562 - auc: 0.5328 - val_loss: 8.1635e-09 - val_accuracy: 0.9557 - val_auc: 0.5440
Epoch 5/50
5783/5783 [==============================] - 117s 20ms/step - loss: 4.4785e-08 - accuracy: 0.9562 - auc: 0.5482 - val_loss: 7.6538e-09 - val_accuracy: 0.9557 - val_auc: 0.5192
Epoch 6/50
5783/5783 [==============================] - 117s 20ms

TypeError: unsupported operand type(s) for -: 'builtin_function_or_method' and 'builtin_function_or_method'

32 BS: 170s per epoch

128 BS: 150s per epoch

256 BS: 125s per epoch

1024 BS: 95s per epoch

# Tests and predictions

In [ ]:
plt.title("Loss with epochs")
plt.plot(history.history["loss"], label='train')
plt.plot(history.history["val_loss"], label='val')
plt.legend()
plt.show()

In [ ]:
y_preds = model.predict(X_test.values, batch_size=1024).reshape(-1)

In [ ]:
accuracy_score(y_test.values, y_preds.round())

Muito sinal classificado como bkgd !!! 10 x mais do que sinal corretamente classificado

In [ ]:
cm = confusion_matrix(y_test.values, y_preds.round(), normalize="all")

ax = plt.subplot()
sns.heatmap(cm, annot=True, ax=ax)
ax.set_title("Confusion Matrix")
ax.set_xlabel("Predicted Labels")
ax.set_ylabel("True Labels")
ax.xaxis.set_ticklabels(["bkgd", "fcnc"])
ax.yaxis.set_ticklabels(["bkgd", "fcnc"])
plt.show()

# Monte Carlo Dropout

In [ ]:
mcpreds = []
num_preds = 100

for _ in tqdm(range(num_preds), total=num_preds, desc="MCDropout"):
    mcpreds.append(model.predict(X_test.values, batch_size=1024).reshape(-1))

In [ ]:
mcpreds = np.array(mcpreds).mean(axis=0).round()

In [ ]:
accuracy_score(y_test.values, mcpreds)

In [ ]:
cm = confusion_matrix(y_test.values, mcpreds, normalize="all")

ax = plt.subplot()
sns.heatmap(cm, annot=True, ax=ax)
ax.set_title("Confusion Matrix")
ax.set_xlabel("Predicted Labels")
ax.set_ylabel("True Labels")
ax.xaxis.set_ticklabels(["bkgd", "fcnc"])
ax.yaxis.set_ticklabels(["bkgd", "fcnc"])
plt.show()